In [1]:
from pymfe.mfe import MFE
import pandas as pd
import tensorflow as tf
import numpy as np
import os
import optuna
from tensorflow.keras.models import Sequential
from keras.src.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from scipy import stats
import ast
import time

C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [40]:
df = pd.read_csv(r"D:\MATTHEW\UROPS\Datasets\All Datasets\Batch 1\Formatted Datasets\100_plants_margin.csv")

In [42]:
X = df.iloc[:, :-1].values  # All columns except the last one for features
y = df.iloc[:, -1].values   # Last column for target

In [5]:
mfe = MFE(groups=['general', 'statistical', 'info-theory'])
mfe.fit(X, y)
ft = mfe.extract()

C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pymfe\_internal.py:1568: UserWarning: It is not possible make equal discretization
  warnings.warn("It is not possible make equal discretization")
C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pymfe\_internal.py:1281: UserWarning:  * Something went wrong while precomputing 'precompute_can_cors'. Will ignore this method. Error message:
TypeError("OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'").
  warnings.warn(
C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pymfe\_internal.py:606: RuntimeWarning: Can't extract feature 'can_cor'.
 Exception message: TypeError("OneHotEncoder.__init__() got an unexpected keyword argument '

In [41]:
df

,V6,V46,V29,V10,V17,V44,V33,V57,V5,V59,class
0,0.154300,0.000000,0.011719,0.023438,0.015625,0.000000,0.025391,0.005859,0.000000,0.003906,0
1,0.093750,0.000000,0.050781,0.011719,0.023438,0.001953,0.054688,0.025391,0.000000,0.041016,0
2,0.025391,0.001953,0.019531,0.019531,0.007812,0.007812,0.021484,0.015625,0.003906,0.031250,0
3,0.150390,0.003906,0.021484,0.017578,0.003906,0.003906,0.017578,0.005859,0.001953,0.015625,0
4,0.134770,0.000000,0.029297,0.013672,0.017578,0.000000,0.042969,0.005859,0.000000,0.021484,1
...,...,...,...,...,...,...,...,...,...,...,...
1595,0.000000,0.035156,0.001953,0.005859,0.000000,0.009766,0.001953,0.005859,0.021484,0.001953,1
1596,0.000000,0.017578,0.000000,0.000000,0.001953,0.011719,0.000000,0.000000,0.037109,0.052734,1
1597,0.000000,0.003906,0.074219,0.000000,0.001953,0.015625,0.000000,0.000000,0.007812,0.138670,1
1598,0.042969,0.027344,0.007812,0.021484,0.011719,0.019531,0.011719,0.013672,0.035156,0.011719,1


In [2]:
def get_mfe(input_dir):
    # input_dir = dir + '\\' + 'Formatted Datasets'
    result_general = {}
    result_statistical = {}
    result_info_theory = {}
    for file in os.listdir(input_dir):
        df = pd.read_csv(input_dir + '\\' + file)
        X = df.iloc[:, :-1] # All columns except the last one for features
        y = df.iloc[:, -1].values   # Last column for target
        numeric_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
        binary_cols = [col for col in numeric_cols if X[col].nunique() == 2]

        # Remove binary columns from numeric list
        numeric_cols = [col for col in numeric_cols if col not in binary_cols]

        X_binary = X[binary_cols].values
        X_numeric = X[numeric_cols].values

        mfe_general = MFE(groups='general')
        mfe_info_theory = MFE(groups='info-theory')

        mfe_general.fit(X.values, y)
        mfe_info_theory.fit(X.values, y)

        ft_general = mfe_general.extract()
        ft_info_theory = mfe_info_theory.extract()

        if numeric_cols != []:
                mfe_statistical = MFE(groups='statistical')
                mfe_statistical.fit(X_numeric, y)
                ft_statistical = mfe_statistical.extract()
                result_statistical[file] = ft_statistical
        
        result_general[file] = ft_general
        result_info_theory[file] = ft_info_theory
        

    return result_general, result_statistical, result_info_theory

In [ ]:
# Load your dataset
df = pd.read_csv(r"D:\MATTHEW\UROPS\Datasets\All Datasets\All Formatted Valid Datasets\Fertility.csv")
X = df.iloc[:, :-1]  # All columns except the last one for features
y = df.iloc[:, -1].values   # Last column for target

# Identify numeric (integer/float) and binary (only 0/1) features
numeric_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
binary_cols = [col for col in numeric_cols if X[col].nunique() == 2]

        # Remove binary columns from numeric list
numeric_cols = [col for col in numeric_cols if col not in binary_cols]

        # Create separate DataFrames for numeric and binary features
X_numeric = X[numeric_cols].values if numeric_cols else None
X_binary = X[binary_cols].values if binary_cols else None

In [9]:
binary_cols

['child_diseases', 'accident', 'surgical_intervention']

In [4]:
all_mfe_general, all_mfe_statistical, all_mfe_info_theory = get_mfe(r'D:\MATTHEW\UROPS\Datasets\All Datasets\All Formatted Valid Datasets')

C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pymfe\_internal.py:1568: UserWarning: It is not possible make equal discretization
  warnings.warn("It is not possible make equal discretization")
C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pymfe\_internal.py:1568: UserWarning: It is not possible make equal discretization
  warnings.warn("It is not possible make equal discretization")
C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pymfe\_internal.py:1281: UserWarning:  * Something went wrong while precomputing 'precompute_can_cors'. Will ignore this method. Error message:
TypeError("OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'").
  warnings.warn(
C:\Users\Matthew W

In [7]:
rows = []
for file_name, (columns, values) in all_mfe_statistical.items():
    row_dict = {"file_name": file_name}  # Store the filename
    row_dict.update(dict(zip(columns, values)))  # Map column names to values
    rows.append(row_dict)

# Create a DataFrame
mfe_df_statistical = pd.DataFrame(rows)

In [8]:
mfe_df_statistical

,file_name,can_cor.mean,can_cor.sd,cor.mean,cor.sd,cov.mean,cov.sd,eigenvalues.mean,eigenvalues.sd,g_mean.mean,...,sd_ratio,skewness.mean,skewness.sd,sparsity.mean,sparsity.sd,t_mean.mean,t_mean.sd,var.mean,var.sd,w_lambda
0,accident.csv,NaN,NaN,0.101797,NaN,4.591803e+01,NaN,5.602416e+02,4.743684e+02,52.019625,...,NaN,0.006252,0.074634,0.010902,0.005052,56.762931,18.610319,NaN,NaN,NaN
1,Activity recognition with healthy older people...,NaN,NaN,0.125604,0.171509,1.541301e-01,2.724892e-01,4.708247e+00,8.466976e+00,NaN,...,NaN,-0.067399,0.569779,0.052162,0.091924,124.358483,352.695747,NaN,NaN,NaN
2,Airline Passenger Satisfaction.csv,NaN,NaN,0.144996,0.189817,3.312366e+01,1.824110e+02,5.919098e+04,2.503391e+05,49.976234,...,NaN,0.540183,2.400224,0.132475,0.070863,61.525129,237.934817,NaN,NaN,NaN
3,air_quality.csv,NaN,NaN,0.481496,0.147376,2.055998e+02,3.611263e+02,2.804507e+03,7.806803e+03,NaN,...,NaN,1.101498,0.961291,0.002278,0.001493,76.383628,161.473183,NaN,NaN,NaN
4,apple_quality.csv,NaN,NaN,0.152056,0.088178,5.119546e-01,3.254920e-01,3.387783e+00,1.801411e+00,NaN,...,NaN,0.001650,0.060423,0.000000,0.000000,0.016899,0.707188,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,Wilt.csv,NaN,NaN,0.261211,0.258804,6.238303e+02,9.067200e+02,5.037585e+03,9.244037e+03,188.002699,...,NaN,2.865844,3.255249,0.000030,0.000019,190.157552,175.202043,NaN,NaN,NaN
93,wine.csv,NaN,NaN,0.318975,0.187608,4.661224e+01,2.375464e+02,8.999928e+03,3.239279e+04,70.138641,...,NaN,0.222864,0.586599,0.006233,0.005545,73.899579,222.684285,NaN,NaN,NaN
94,winequality-red.csv,NaN,NaN,0.128541,0.169982,3.713220e+13,2.664985e+14,1.177522e+26,3.905399e+26,7.018844,...,NaN,6.454175,8.397961,0.009608,0.004191,7.202812,11.544225,NaN,NaN,NaN
95,winequality-white.csv,NaN,NaN,0.089940,0.121488,2.000318e+13,1.409504e+14,1.332767e+26,4.420288e+26,17.603078,...,NaN,9.762680,17.578012,0.008389,0.004495,18.274199,41.208374,NaN,NaN,NaN


In [ ]:
nan_columns = mfe_df_statistical.columns[mfe_df_statistical.isna().all()].tolist()
print("Columns with NaN values:", nan_columns)

Columns with NaN values: []


In [9]:
len(nan_columns)

33

In [42]:
mfe_df_statistical[mfe_df_statistical['file_name'] == 'iris.csv']

,file_name,can_cor.mean,can_cor.sd,cor.mean,cor.sd,cov.mean,cov.sd,eigenvalues.mean,eigenvalues.sd,g_mean.mean,...,sd_ratio,skewness.mean,skewness.sd,sparsity.mean,sparsity.sd,t_mean.mean,t_mean.sd,var.mean,var.sd,w_lambda
48,iris.csv,NaN,NaN,0.612493,0.325409,0.503962,0.443638,0.947728,1.687481,3.205272,...,NaN,0.088384,0.39513,0.031306,0.013219,3.434167,1.915953,NaN,NaN,NaN


In [38]:
mfe_df_statistical[mfe_df_statistical['sd_ratio'].notna()]

,file_name,can_cor.mean,can_cor.sd,cor.mean,cor.sd,cov.mean,cov.sd,eigenvalues.mean,eigenvalues.sd,g_mean.mean,...,sd_ratio,skewness.mean,skewness.sd,sparsity.mean,sparsity.sd,t_mean.mean,t_mean.sd,var.mean,var.sd,w_lambda


In [32]:
mfe_df = mfe_df.dropna(axis=1, how='all')

In [11]:
mfe_df_info_theory.to_csv(r'D:\MATTHEW\UROPS\Datasets\All Datasets\All Datasets Info-Theory Meta-Features.csv', index=False)

In [2]:
def create_mlp(X, y, num_layers, num_neurons):
    model = Sequential()

    model.add(Dense(num_neurons[0], input_dim = X.shape[1], activation = 'relu'))

    for i in range(1, num_layers):
        model.add(Dense(num_neurons[i], activation = 'relu'))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [3]:
def objective(trial, df):
    X = df.iloc[:, :-1]  # All columns except the last one for features
    y = df.iloc[:, -1].values   # Last column for target
    # numeric_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
    # binary_cols = [col for col in numeric_cols if X[col].nunique() == 2]

    #     # Remove binary columns from numeric list
    # numeric_cols = [col for col in numeric_cols if col not in binary_cols]
    # X_binary = X[binary_cols].values
    # X_numeric = X[numeric_cols].values

    # mfe_general = MFE(groups='general')
    # mfe_info_theory = MFE(groups='info-theory')

    # mfe_general.fit(X.values, y)
    # mfe_info_theory.fit(X.values, y)

    # ft_general_names, ft_general_values = mfe_general.extract()
    # ft_info_theory_names, ft_info_theory_values = mfe_info_theory.extract()

    # if numeric_cols != []:
    #         mfe_statistical = MFE(groups='statistical')
    #         mfe_statistical.fit(X_numeric, y)
    #         ft_statistical_names, ft_statistical_values = mfe_statistical.extract()
    #         meta_features_statistical = dict(zip(ft_statistical_names, ft_statistical_values))
    #         trial.set_user_attr("meta_features_statistical", meta_features_statistical)
    # meta_features_general = dict(zip(ft_general_names, ft_general_values))
    # meta_features_info_theory = dict(zip(ft_info_theory_names, ft_info_theory_values))
    # Sample hyperparameters
    num_layers = trial.suggest_int("num_layers", 1, 3)
    num_neurons = [trial.suggest_int(f"neurons_layer_{i+1}", 1, 16) for i in range(num_layers)]
    # trial.set_user_attr("meta_features_general", meta_features_general)
    # trial.set_user_attr("meta_features_info_theory", meta_features_info_theory)

    # Train your MLP and return validation loss or accuracy
    val_accuracy = test_mlp_cv(X.values, y, num_layers, num_neurons)
    
    return val_accuracy  # Maximizing accuracy

In [4]:
def test_mlp(X, y, num_layers, num_neurons):
    model = create_mlp(X, y, num_layers, num_neurons)
    history = model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2, verbose=0)
    val_accuracy = history.history["val_accuracy"][-1]  # Extract last validation accuracy
    return val_accuracy  # Return a float (Optuna requires this!)

In [5]:
def test_mlp_cv(X, y, num_layers, num_neurons, cv_folds = 10):
    kf = KFold(n_splits=cv_folds)
    accuracies = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Train your model
        val_accuracy = test_mlp(X_train, y_train, num_layers, num_neurons)
        
        # Validate on test set and store accuracy
        accuracies.append(val_accuracy)
    
    return np.mean(accuracies)  # Return average accuracy across folds

In [8]:
def smbo(dir):
    result = {}
    dir = dir + '\\' + 'Formatted Datasets'
#     s3 = boto3.client('s3')

#     # Your project storage bucket name
#     bucket_name = "rg-dsa32882420wija-dsa32882420wija-499"
#     input_dir = dir + '/Formatted Datasets'
# #     split_dir = 's3://rg-dsa32882420wija-dsa32882420wija-499/' + dir + '/Splitted Datasets'
#     bucket_path = 's3://rg-dsa32882420wija-dsa32882420wija-499'

#     # List objects in the folder
#     response = s3.list_objects_v2(Bucket=bucket_name, Prefix=input_dir)

#     # Extract file names
#     files = [obj['Key'] for obj in response.get('Contents', [])]
    
    for file in os.listdir(dir):
        print(file)
        start = time.time()
        df = pd.read_csv(dir + '\\' + file)
        
        # Run the optimization
        study = optuna.create_study(direction="maximize")
        study.optimize(lambda trial: objective(trial, df), n_trials=50)

        best_hyperparams = study.best_params
        best_accuracy = study.best_value
        end = time.time()
        runtime = end - start
        # filename = file.split('/')[3]
        
        result[file] = {'best_hp' : best_hyperparams, 'best_accuracy' : best_accuracy, 'runtime' : runtime}
    return result

    # Print results
#     print("Best Hyperparameters:", best_hyperparams)
#     print("Best Accuracy:", best_accuracy)

In [9]:
for i in [1]:
    print(f'Train Batch {i}')
    dir = r'D:\MATTHEW\UROPS\Datasets\All Datasets\Train Datasets\Train Batch ' + str(i)
    print(dir)
    result = smbo(dir)
    result_df = pd.DataFrame([
    {
        'filename': filename,
        'best_accuracy': data['best_accuracy'],
        'num_layers': data['best_hp']['num_layers'],
        'runtime' : data['runtime'],
        
        **{f'neurons_layer_{i+1}': data['best_hp'].get(f'neurons_layer_{i+1}', None)
           for i in range(data['best_hp']['num_layers'])}  # Handle variable layers
    }
    for filename, data in result.items()
])
    result_df.fillna(0, inplace=True)
    result_df['neurons_layer_2'] = result_df['neurons_layer_2'].astype(int)
    result_df['neurons_layer_3'] = result_df['neurons_layer_3'].astype(int)
    result_df.to_csv(r'D:\MATTHEW\UROPS\Datasets\All Datasets\SMBO Results' + '\\' +  f'Train Batch {i} SMBO Results.csv', index=False)

[I 2025-03-08 20:39:16,907] A new study created in memory with name: no-name-c87e59e9-e3fc-48ab-bedd-e12dab70b6b1


Train Batch 1
D:\MATTHEW\UROPS\Datasets\All Datasets\Train Datasets\Train Batch 1
accident.csv


C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[I 2025-03-08 20:39:49,466] Trial 0 finished with value: 0.26285714749246836 and parameters: {'num_layers': 2, 'neurons_layer_1': 12, 'neurons_layer_2': 5}. Best is trial 0 with value: 0.26285714749246836.
[I 2025-03-08 20:40:22,152] Trial 1 finished with value: 0.29142857510596515 and parameters: {'num_layers': 3, 'neurons_layer_1': 13, 'neurons_layer_2': 11, 'neurons_layer_3': 9}. Best is trial 1 with value: 0.29142857510596515.
[I 2025-03-08 20:40:54,612] Trial 2 finished with value: 0.2828571455553174 and parameters: {'num_layers': 3, 'neurons_layer_1

Activity recognition with healthy older people using a batteryless wearable sensor.csv


[I 2025-03-08 22:41:55,479] A new study created in memory with name: no-name-55458b12-f7ed-43ac-8d64-4cef3c1e5f77
C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[I 2025-03-08 22:59:36,006] Trial 0 finished with value: 0.6739860266447067 and parameters: {'num_layers': 2, 'neurons_layer_1': 12, 'neurons_layer_2': 3}. Best is trial 0 with value: 0.6739860266447067.
[I 2025-03-08 23:17:24,798] Trial 1 finished with value: 0.9367701649665833 and parameters: {'num_layers': 1, 'neurons_layer_1': 10}. Best is trial 1 with value: 0.9367701649665833.
[I 2025-03-08 23:36:13,605] Trial 2 finished with value: 0.62

Airline Passenger Satisfaction.csv


C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[I 2025-03-09 06:08:43,731] Trial 0 finished with value: 0.1624362885951996 and parameters: {'num_layers': 2, 'neurons_layer_1': 2, 'neurons_layer_2': 1}. Best is trial 0 with value: 0.1624362885951996.
[I 2025-03-09 06:26:42,241] Trial 1 finished with value: 0.8195489823818207 and parameters: {'num_layers': 2, 'neurons_layer_1': 3, 'neurons_layer_2': 16}. Best is trial 1 with value: 0.8195489823818207.
[I 2025-03-09 06:44:29,147] Trial 2 finished with value: 0.8192452251911163 and parameters: {'num_layers': 1, 'neurons_layer_1': 13}. Best is trial 1 with

KeyboardInterrupt: 

In [51]:
result_df = pd.read_csv(r"D:\MATTHEW\UROPS\Datasets\All Datasets\Batch 1\Batch 1 Results.csv")
result_df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,100_plants_margin.csv,0.486875,0.560000,0.472500,0.486250,0.532500,0.545000,0.471875,0.481875,0.476250,0.488750,0.501250,0.525000,0.493750,0.530625
1,100_plants_shape.csv,0.475000,0.476250,0.475000,0.475000,0.476250,0.475000,0.475000,0.475000,0.475000,0.476250,0.475000,0.475000,0.475000,0.476250
2,100_plants_texture.csv,0.488074,0.566950,0.473074,0.500594,0.541297,0.568840,0.470574,0.482484,0.482449,0.503746,0.505597,0.542001,0.498781,0.564450
3,amazon_review.csv,0.480667,0.486000,0.488000,0.490000,0.480667,0.483333,0.472667,0.465333,0.476667,0.466667,0.465333,0.485333,0.464667,0.481333
4,analcatdata_authorship.csv,0.570664,0.816318,0.502777,0.605030,0.630604,0.810543,0.468712,0.468712,0.477284,0.542515,0.656620,0.584044,0.662052,0.820382
5,AP_Colon_Kidney.csv,0.450000,0.850000,0.434615,0.500000,0.509615,0.844231,0.480769,0.484615,0.453846,0.451923,0.453846,0.488462,0.448077,0.846154
6,ar4.csv,0.425000,0.475000,0.450000,0.450000,0.400000,0.550000,0.400000,0.400000,0.450000,0.500000,0.400000,0.425000,0.425000,0.625000
7,ar6.csv,0.466667,0.466667,0.433333,0.300000,0.433333,0.566667,0.266667,0.400000,0.300000,0.333333,0.400000,0.533333,0.300000,0.333333
8,banana_quality.csv,0.833000,0.960125,0.657750,0.830625,0.963875,0.971625,0.677250,0.604250,0.561750,0.727500,0.874250,0.972750,0.963625,0.973250
9,Banknote.csv,0.615574,0.975410,0.550000,0.751639,0.873770,0.995902,0.550820,0.667213,0.529508,0.747541,0.784426,0.944262,0.991803,1.000000
